# Generating Images of Digits from Text Prompts

This notebook provides you with a complete code example to generate MNIST digits from text prompts. The `train_text_prompts.txt` and `test_text_prompts.txt` files provide the text data used for training and testing.

## Loading the MNIST Dataset with PyTorch

Implement the normalization of the images ...

In [ ]:
from torchvision.transforms import Compose, Normalize, ToTensor

trans = Compose([ToTensor(), Normalize(mean=[0.5], std=[0.5], inplace=True)])

...create the dataset...

In [ ]:
import json

with open("mnist_sentences.json", "r") as file:
    filenames_sentences = json.load(file)

In [ ]:
for filename, sentence in filenames_sentences.items():
    print(f"{filename} : {sentence}")

In [ ]:
import os

if not os.path.exists("MNIST_dataset"):
    os.system("git clone https://github.com/DeepTrackAI/MNIST_dataset")
MNIST_path = os.path.join("MNIST_dataset", "mnist", "train")
MNIST_images_files = sorted(os.listdir(MNIST_path))

In [ ]:
import matplotlib.pyplot as plt

images, digits, sentences = [], [],[]
for filename in MNIST_images_files:
    image = plt.imread(os.path.join(MNIST_path, filename))
    images.append(trans(image))
    
    digit = int(filename[0])
    digits.append(digit)

    sentence = filenames_sentences.get(filename, "")
    sentences.append(sentence)

trainset = list(zip(images, digits, sentences))


... and plot some of the transformed MNIST digits.

In [ ]:
import torch

fig, axs = plt.subplots(1, 8, figsize=(15, 3))
for ax in axs.ravel():
    img, label, _ = trainset[torch.randint(0, len(trainset), (1,)).squeeze()]
    ax.imshow(img.squeeze(), cmap="gray")
    ax.set_title(f"Label: {label}", fontsize=16)
    ax.axis("off")
plt.tight_layout()
plt.show()


## Implementing the Forward Process

Define the device on which the computations are performed ...

In [ ]:
import torch

def get_device():
    """Select device where to perform computations."""
    if torch.cuda.is_available():
        return torch.device("cuda:0")
    # elif torch.backends.mps.is_available():
    #    return torch.device("mps")
    else:
        return torch.device("cpu")

In [ ]:
device = get_device()

In [ ]:
print(device)

... implement the forward diffusion process ...

In [ ]:
class Diffusion:
    """Denoising diffusion probabilstic model (DDPM)."""
    
    def __init__(self, steps=1000, beta_start=1e-4, beta_end=0.02,
                 device=device):
        """Initialize the diffusion model."""
        self.steps, self.device = steps, device
        self.beta = torch.linspace(beta_start, beta_end, self.steps).to(device)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)

    def forward_diffusion(self, x, t):
        """Implement the forward diffusion process."""
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t])[:, None, None, None]
        sqrt_one_minus_alpha_bar = \
            torch.sqrt(1 - self.alpha_bar[t])[:, None, None, None]
        noise = torch.randn_like(x)
        return sqrt_alpha_bar * x + sqrt_one_minus_alpha_bar * noise, noise

... sampling images in the forward diffusion process ..

In [ ]:
diffusion = Diffusion(steps=401, beta_start=0.0001, beta_end=0.02)

clean_image, label, _ = trainset[torch.randint(0, len(trainset), (1,)).squeeze()]

steps = [0, 100, 200, 300, 400] 
noisy_images = []
for t in steps:
    noisy_image, noise = diffusion.forward_diffusion(
        x=clean_image[None, ...].to(device), t=torch.tensor([t]).to(device),
    )
    noisy_images.append(noisy_image)

... and visualize the noisy digits generated in the forward diffusion process.

In [ ]:
fig, axs = plt.subplots(1, len(steps))
for i, ax in enumerate(axs.flatten()):
    ax.imshow(noisy_images[i].cpu().numpy().squeeze(), cmap="gray")
    ax.set_title(f"t = {steps[i]}", fontsize=10)
    ax.axis("off")
plt.tight_layout()
plt.show()

## Implementing the Reverse Diffusion Process

Update the `Diffusion` class to implement the reverse diffusion process.

In [ ]:
from tqdm import tqdm

class Diffusion:
    """Denoising diffusion probabilstic model (DDPM)."""
    
    def __init__(self, steps=1000, beta_start=1e-4, beta_end=0.02, img_size=28, 
                 device=device):
        """Initialize the diffusion model."""
        self.steps, self.img_size, self.device = steps, img_size, device
        self.beta = torch.linspace(beta_start, beta_end, self.steps).to(device)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)

    def forward_diffusion(self, x, t):
        """Implement the forward diffusion process."""
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t])[:, None, None, None]
        sqrt_one_minus_alpha_bar = \
            torch.sqrt(1 - self.alpha_bar[t])[:, None, None, None]
        noise = torch.randn_like(x)
        return sqrt_alpha_bar * x + sqrt_one_minus_alpha_bar * noise, noise
    
    def reverse_diffusion(self, model, n_images, n_channels, pos_enc_dim, 
                          pos_enc_func, fix_noise=None, save_time_steps=None, 
                          context=None, guidance_strength=None):
        """Reverse diffusion process"""
        with torch.no_grad():
            if fix_noise is not None:
                x = fix_noise.to(self.device)
            else:
                x = torch.randn(
                    (n_images, n_channels, self.img_size, self.img_size)
                ).to(self.device)
            
            denoised_images = []
            for i in tqdm(reversed(range(0, self.steps)),
                          desc="U-Net inference", total=self.steps):
                t = (torch.ones(n_images) * i).long()
                t_pos_enc = (pos_enc_func(t.unsqueeze(1), pos_enc_dim)
                             .to(self.device))
                if context is None:
                    predicted_noise = model(x=x, t=t_pos_enc)
                else:
                    conditional_pred = model(x=x, t=t_pos_enc, context=context)
                    unconditional_pred = model(x=x, t=t_pos_enc, context=None)
                    predicted_noise = torch.lerp(unconditional_pred,
                                                 conditional_pred,
                                                 guidance_strength)
                alpha = self.alpha[t][:, None, None, None]
                alpha_bar = self.alpha_bar[t][:, None, None, None]
                noise = torch.randn_like(x) if i > 0 else torch.zeros_like(x)
                x = (1 / torch.sqrt(alpha) * (x - ((1 - alpha) 
                     / torch.sqrt(1 - alpha_bar)) * predicted_noise) 
                     + torch.sqrt(1-alpha) * noise)
                if i in save_time_steps: denoised_images.append(x)

            denoised_images = torch.stack(denoised_images)
            denoised_images = denoised_images.swapaxes(0, 1)
            return denoised_images

## Defining the Position Encoding Function

Implement the position encoding function ...

In [ ]:
def positional_encoding(t, enc_dim):
    """Encode position information with a sinusoid."""
    scaled_positions = torch.arange(0, enc_dim, 2).float() / enc_dim
    frequency = 10000 ** scaled_positions
    inverse_frequency = (1.0 / frequency).to(t.device)
    angle = t.repeat(1, enc_dim // 2) * inverse_frequency
    pos_enc_a, pos_enc_b = torch.sin(angle), torch.cos(angle)
    pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
    return pos_enc

... sinusoidal position encodings for different time steps ...

In [ ]:
pos_enc_dim = 256

pos_encs = []
for i in range(0, 100):
    t = torch.tensor([i])
    pos_enc = positional_encoding(t, pos_enc_dim)
    pos_encs.append(pos_enc.squeeze())
pos_encs = torch.stack(pos_encs)

... and visualize the position encodings.

In [ ]:
fig = plt.figure()
plt.imshow(pos_encs.cpu().numpy())
plt.xlabel("Encoding dimension")
plt.ylabel("Time step (t)")
plt.show()

## Defining a Custom Tokenizer and Text Encoder

Implement the function to tokenize the text ...

In [ ]:
import contractions, re, spacy, unicodedata

tokenizers = {"eng": spacy.blank("en"), "spa": spacy.blank("es")}

regular_expression = r"^[a-zA-Z0-9áéíóúüñÁÉÍÓÚÜÑ.,!?¡¿/:()]+$"
pattern = re.compile(unicodedata.normalize("NFC", regular_expression))

def tokenize(text, lang="eng"):
    """Tokenize text."""
    swaps = {"’": "'", "‘": "'", "“": '"', "”": '"', "´": "'", "´´": '"'}
    for old, new in swaps.items():
        text = text.replace(old, new)
    text = contractions.fix(text) if lang == "eng" else text
    tokens = tokenizers[lang](text)
    return [token.text for token in tokens if pattern.match(token.text)]

In [ ]:
class Vocab:
    """Vocabulary as callable dictionary."""
    
    def __init__(self, vocab_dict, unk_token="<unk>"):
        """Initialize vocabulary"""
        self.vocab_dict, self.unk_token = vocab_dict, unk_token
        self.default_index = vocab_dict.get(unk_token, -1)
        self.index_to_token = {idx: token for token, idx in vocab_dict.items()}
        
    def __call__(self, token_or_tokens):
        """Return the index(es) for given token or list of tokens."""
        if not isinstance(token_or_tokens, list):
            return self.vocab_dict.get(token_or_tokens, self.default_index)
        else:
            return [self.vocab_dict.get(token, self.default_index) 
                    for token in token_or_tokens]
    
    def set_default_index(self, index):
        """Set default index for unknown tokens."""
        self.default_index = index

    def lookup_token(self, index_or_indices):
        """Retrieve token corresponding to given index or list of indices."""
        if not isinstance(index_or_indices, list):
            return self.index_to_token.get(int(index_or_indices), self.unk_token)
        else:
            return [self.index_to_token.get(int(index), self.unk_token) 
                    for index in index_or_indices]

    def get_itos(self):
        """Return a list of tokens ordered by their index."""
        itos = [None] * len(self.index_to_token)
        for index, token in self.index_to_token.items():
            itos[index] = token
        return itos
        
    def __iter__(self):
        """Iterate over the tokens in the vocabulary."""
        return iter(self.vocab_dict)

    def __len__(self):
        """Return the number of tokens in the vocabulary."""
        return len(self.vocab_dict)
    
    def __contains__(self, token):
        """Check if a token is in the vocabulary."""
        return token in self.vocab_dict

In [ ]:
from collections import Counter

def build_vocab_from_iterator(iterator, specials=None, min_freq=1):
    """Build vocabulary from an iterator over tokenized sentences."""
    token_freq = Counter(token for tokens in iterator for token in tokens)
    vocab, index = {}, 0
    if specials: 
        for token in specials: 
            vocab[token] = index
            index += 1
    for token, freq in token_freq.items():
        if freq >= min_freq:
            vocab[token] = index
            index += 1
    return vocab

... build a custom vocabulary ...

In [ ]:
def sentence_iterator(dataset):
    """Iterate over the dataset."""
    for sentence in dataset:
        yield tokenize(sentence)

with open("test_sentences.txt", "r") as file:
    test_sentences = [line.strip() for line in file.readlines()]

vocab_dict = build_vocab_from_iterator(
    sentence_iterator(sentences + test_sentences), 
    specials=["<unk>", "<sos>", "<eos>", "<pad>"], min_freq=1,
)
vocab = Vocab(vocab_dict, unk_token="<unk>")
vocab.set_default_index(vocab(vocab.unk_token))

... check the vocabulary and string-to-numerical indices mapping ...

In [ ]:
for word, index in vocab_dict.items(): print(f"{word}: {index}")

... write a function to implemet a function to pad and process ...

In [ ]:
def pad_and_process(texts, vocab=vocab, max_token_length=77):
    """Tokenize a sentence."""
    batch_tokens = []
    for text_prompt in texts:
        tokens = ([vocab("<sos>")] + 
                  [vocab(token) for token in tokenize(text_prompt)] + 
                  [vocab("<eos>")])

        if len(tokens) > max_token_length:
            tokens = tokens[:max_token_length]
        else:
            tokens += [vocab("<pad>")] * (max_token_length - len(tokens))
        
        tokens = torch.tensor(tokens, dtype=torch.long)
        batch_tokens.append(tokens)
    return torch.stack(batch_tokens)

... observe a tokenized version of an example text prompt ...

In [ ]:
import random

_, _, example_text = random.choice(trainset)
tokens = pad_and_process([example_text])
print(example_text), print(tokens.shape), print(tokens);

... implement a class for a custom text encoder ...

In [ ]:
import torch.nn as nn

class TextEncoder(nn.Module):
    """Text encoder."""

    def __init__(self, max_token_length, vocab_size, embedding_dim, num_heads):
        """Initialize the text encoder module."""
        super(TextEncoder, self).__init__()
        
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.position_encoding = positional_encoding(
            torch.arange(0, max_token_length).unsqueeze(1), embedding_dim,
        )

        self.self_attention = nn.MultiheadAttention(
            embedding_dim, num_heads=num_heads, batch_first=True,
        )
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)

        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim), nn.GELU(),
            nn.Linear(embedding_dim, embedding_dim),
        )

    def forward(self, tokens):
        """Forward pass of text encoder module."""
        token_embeddings = self.token_embedding(tokens)
        position_encodings = (self.position_encoding
                              .repeat(tokens.size(0), 1, 1).to(tokens.device))

        token_embeddings_with_pos = token_embeddings + position_encodings

        normalized_embeddings = self.layer_norm1(token_embeddings_with_pos)
        attention_output, _ = self.self_attention(
            query=normalized_embeddings, 
            key=normalized_embeddings, 
            value=normalized_embeddings,
        )
        attention_output = attention_output + normalized_embeddings

        residual_input = attention_output
        normalized_attention_output = self.layer_norm2(attention_output)
        feed_forward_output = self.feed_forward(normalized_attention_output)
        residual_output = feed_forward_output + residual_input

        return residual_output

... instantiate a custom text encoder ...

In [ ]:
custom_text_encoder = TextEncoder(max_token_length=77, vocab_size=len(vocab),
                                  embedding_dim=768, num_heads=4).to(device)

... and check the shape of example text prompts.

In [ ]:
_, _, example_text = random.choice(trainset)  ### ADD
tokens = pad_and_process([example_text])
text_embedding = custom_text_encoder(tokens.to(device))

print(f"tokens shape: {tokens.shape}")
print(f"text embeddings shape: {text_embedding.shape}")

## Defining the Conditional Attention U-Net

In [ ]:
import deeplay as dl

pos_enc_dim = 256

unet_template = dl.AttentionUNet(in_channels=1, channels=[32, 64, 128], 
                                 base_channels=[256, 256], 
                                 channel_attention=[True, True, True], 
                                 out_channels=1, 
                                 position_embedding_dim=pos_enc_dim, 
                                 num_classes=10, context_embedding_dim=768)
unet = unet_template.create()
unet.to(device);

In [ ]:
print(unet)

## Training the Diffusion Model

Define the data loader ...

In [ ]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset=trainset, batch_size=128, shuffle=True)

... define the loss function ...

In [ ]:
criterion = torch.nn.MSELoss()

... define the optimizer ...

In [ ]:
optimizer = torch.optim.AdamW(list(unet.parameters())
                              + list(custom_text_encoder.parameters()), 
                              lr=1e-4)

...  instantiate the diffusion class for training ...

In [ ]:
diffusion = Diffusion(steps=1000, beta_start=1e-4, beta_end=0.02, img_size=28)

...implement a function to prepare the data ...

In [ ]:
def prepare_data(image, steps=1000, device=device):
    """Prepare data."""
    batch_size = image.shape[0]
    t = torch.randint(low=0, high=steps, size=(batch_size,)).to(device)
    image = image.to(device)
    x_t, noise = diffusion.forward_diffusion(image, t)
    t_pos_enc = positional_encoding(t.unsqueeze(1), pos_enc_dim)
    return x_t.to(device), t_pos_enc.to(device), noise.to(device)

... implement the training cycle ...

In [ ]:
import numpy as np
import time
from datetime import timedelta

epochs = 20

n_images = 5
example_sentences = random.sample(test_sentences, n_images)
save_time_steps = [999, 900, 800, 700, 600, 500, 400, 300, 200, 100, 0]
train_loss = []
for epoch in range(epochs):
    unet.train(), custom_text_encoder.train()

    start_time = time.time()
    num_batches = len(loader)

    print("\n" + f"Epoch {epoch + 1}/{epochs}" + "\n" + "_" * 10)
        
    running_loss = 0.0
    for batch_idx, (images, _, sentences) in enumerate(loader, start=0):
        x_t, t, noise = prepare_data(images)

        # Custom Tokenizer.
        tokens = pad_and_process(sentences)
        text_embeddings = custom_text_encoder(tokens.to(device))

        # CLIP.
        """tokens = CLIP_tokenizer(
            sentences, padding="max_length",
            max_length=CLIP_tokenizer.model_max_length, truncation=True,
            return_tensors="pt",
        )
        text_embeddings = CLIP_text_encoder(tokens.input_ids.to(device))[0]"""
        
        context = None if np.random.rand() < 0.1 else text_embeddings

        outputs = unet(x=x_t, t=t, context=context)

        optimizer.zero_grad()
        loss = criterion(outputs, noise)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx + 1}/{num_batches}: "
                  + f"Train loss: {loss.item():.4f}")
        running_loss += loss.item()

    train_loss.append(running_loss / len(loader))
    end_time = time.time()

    print("-" * 10 + "\n" + f"Epoch {epoch + 1}/{epochs} : " 
          + f"Train loss: {train_loss[-1]:.4f}, " 
          + f"Time taken: {timedelta(seconds=end_time - start_time)}")
    
    unet.eval(), custom_text_encoder.eval()

    # Custom Tokenizer.
    tokens = pad_and_process(example_sentences)
    text_embeddings = custom_text_encoder(tokens.to(device))
    
    # CLIP.
    """tokens = CLIP_tokenizer(
        example_texts, padding="max_length",
        max_length=CLIP_tokenizer.model_max_length, truncation=True,
        return_tensors="pt",
    )
    text_embeddings = CLIP_text_encoder(tokens.input_ids.to(device))[0]"""
    
    generated_images = diffusion.reverse_diffusion(
        model=unet, n_images=n_images, n_channels=1, pos_enc_dim=pos_enc_dim,
        pos_enc_func=positional_encoding, save_time_steps=save_time_steps, 
        context=text_embeddings, guidance_strength=3,
    )
    #########################
    fig = plt.figure(figsize=(len(save_time_steps), 1.2 * n_images))
    for idx in range(n_images):
        image_reverse_diff_traj = generated_images[idx]
        for j in range(len(image_reverse_diff_traj)):
            plt.subplot(n_images, len(image_reverse_diff_traj), 
                        idx * len(image_reverse_diff_traj) + j + 1)
            plt.imshow(image_reverse_diff_traj[j]
                       .permute(1, 2, 0).cpu().numpy(), cmap="gray")
            if j == 5: plt.title(example_sentences[idx], fontsize=10)
            plt.axis("off")
    plt.show()
    plt.close()

In [ ]:
unet.eval(), custom_text_encoder.eval()

''' class_labels = random.sample(list(range(10)), n_images)
example_texts = generate_text_prompts(class_labels, test_text_prompts)'''

example_texts = random.sample(test_sentences, n_images)


# Custom Tokenizer.
tokens = pad_and_process(example_texts)
text_embeddings = custom_text_encoder(tokens.to(device))

generated_images = diffusion.reverse_diffusion(
        model=unet, n_images=n_images, n_channels=1, 
        position_encoding_dim=position_encoding_dim,
        position_encoding_function=positional_encoding,
        save_time_steps=save_time_steps, 
        context=text_embeddings, guidance_strength=3,
    )

fig = plt.figure(figsize=(len(save_time_steps), 1.2 * n_images))
for idx in range(n_images):
    image_reverse_diff_traj = generated_images[idx]
    for j in range(len(image_reverse_diff_traj)):
        plt.subplot(n_images, len(image_reverse_diff_traj), 
                    idx * len(image_reverse_diff_traj) + j + 1)
        plt.imshow(image_reverse_diff_traj[j]
                    .permute(1, 2, 0).cpu().numpy(), cmap="gray")
        if j == 5: plt.title(example_texts[idx], fontsize=10)
        plt.axis("off")
plt.savefig(f"fig_10_B1.pdf", bbox_inches="tight")  ### plt.show()

In [ ]:
# raise KeyboardInterrupt  ### REMOVE

## Defining CLIP Tokenizer and CLIP Text Encoder

Import the CLIP tokenizer and CLIP text encoder ...

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer

CLIP_tokenizer = CLIPTokenizer.from_pretrained(
    "openai/clip-vit-large-patch14", torch_dtype=torch.float32,
)
CLIP_text_encoder = CLIPTextModel.from_pretrained(
    "openai/clip-vit-large-patch14", torch_dtype=torch.float32,
).to(device)

... check the tokens and the text embeddings of an example text prompt generated by CLIP ...

In [ ]:
example_text = random.choice(test_sentences)

tokens = CLIP_tokenizer(
    example_text, padding="max_length", 
    max_length=CLIP_tokenizer.model_max_length,
    truncation=True, return_tensors="pt",
)
text_embedding = CLIP_text_encoder(tokens.input_ids.to(device))[0]

print(f"text: {example_text}")
print(f"tokens: {tokens.input_ids}")
print(f"tokens shape: {tokens.input_ids.shape}")
print(f"text embeddings: {text_embedding.shape}")

... and freeze the weights of CLIP.

In [ ]:
for param in CLIP_text_encoder.parameters():
    param.requires_grad = False

### Defining the Conditional Attention U-Net for CLIP

In [ ]:
import deeplay as dl

position_encoding_dim = 256

unet_template = dl.AttentionUNet(  ###
    in_channels=1, channels=[32, 64, 128], base_channels=[256, 256], 
    channel_attention=[True, True, True], out_channels=1, 
    position_embedding_dim=position_encoding_dim, num_classes=10,
    context_embedding_dim=768,
)
unet_clip = unet_template.create()  ### unet.build()
unet_clip.to(device);  ###

print(unet_clip)  ###

## Training the Diffusion Model with CLIP

Remove the `custom_text_encoder` parameters from the optimizer ...

In [ ]:
optimizer_clip = torch.optim.AdamW(unet_clip.parameters(), lr=1e-4)  ###

... remove `custom_text_encoder` and add CLIP to training cycle ...

In [ ]:
import numpy as np
import time
from datetime import timedelta

epochs = 20
n_images = 5

example_texts = random.sample(test_sentences, n_images)

save_time_steps = [999, 900, 800, 700, 600, 500, 400, 300, 200, 100, 0]

train_loss = []
for epoch in range(epochs):
    start_time = time.time()
    num_batches = len(loader)

    print("\n" + f"Epoch {epoch + 1}/{epochs}" + "\n" + "_" * 10)
    
    unet_clip.train() ###, custom_text_encoder.train()
    
    running_loss = 0.0
    for batch_idx, (images, class_labels, text_inputs) in enumerate(loader, start=0):
        x_t, t, noise = prepare_data(images)

        # Custom Tokenizer.
        """tokens = pad_and_process(text_inputs)
        text_embeddings = custom_text_encoder(tokens.to(device))"""

        # CLIP.
        tokens = CLIP_tokenizer(
            text_inputs, padding="max_length",
            max_length=CLIP_tokenizer.model_max_length, truncation=True,
            return_tensors="pt",
        )
        text_embeddings = CLIP_text_encoder(tokens.input_ids.to(device))[0]
        
        context = None if np.random.rand() < 0.1 else text_embeddings
        
        outputs = unet_clip(x=x_t, t=t, context=context)  ###

        optimizer_clip.zero_grad()
        loss = criterion(outputs, noise)
        loss.backward()
        optimizer_clip.step()
        
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx + 1}/{num_batches}: "
                  + f"Train loss: {loss.item():.4f}")
        running_loss += loss.item()

    train_loss.append(running_loss / len(loader))
    end_time = time.time()

    print("-" * 10 + "\n" + f"Epoch {epoch + 1}/{epochs} : " 
          + f"Train loss: {train_loss[-1]:.4f}, " 
          + f"Time taken: {timedelta(seconds=end_time - start_time)}")
    
    unet_clip.eval() ###, custom_text_encoder.eval()

    # example_texts = random.sample(test_sentences, n_images)


    # Custom Tokenizer.
    """tokens = pad_and_process(example_texts)
    text_embeddings = custom_text_encoder(tokens.to(device))"""
    
    # CLIP.
    tokens = CLIP_tokenizer(
        example_texts, padding="max_length",
        max_length=CLIP_tokenizer.model_max_length, truncation=True,
        return_tensors="pt",
    )
    text_embeddings = CLIP_text_encoder(tokens.input_ids.to(device))[0]
    
    generated_images = diffusion.reverse_diffusion(
        model=unet_clip, n_images=n_images, n_channels=1,  ###
        position_encoding_dim=position_encoding_dim,
        position_encoding_function=positional_encoding,
        save_time_steps=save_time_steps, 
        context=text_embeddings, guidance_strength=3,
    )

    fig = plt.figure(figsize=(len(save_time_steps), 1.2 * n_images))
    for idx in range(n_images):
        image_reverse_diff_traj = generated_images[idx]
        for j in range(len(image_reverse_diff_traj)):
            plt.subplot(n_images, len(image_reverse_diff_traj), 
                        idx * len(image_reverse_diff_traj) + j + 1)
            plt.imshow(image_reverse_diff_traj[j]
                       .permute(1, 2, 0).cpu().numpy(), cmap="gray")
            if j == 5: plt.title(example_texts[idx], fontsize=10)
            plt.axis("off")
    plt.show()
    plt.close()

In [ ]:
unet_clip.eval()

''' class_labels = random.sample(list(range(10)), n_images)
example_texts = generate_text_prompts(class_labels, test_text_prompts)'''

example_texts = random.sample(test_sentences, n_images)

# Custom Tokenizer.
# tokens = pad_and_process(example_texts)
# text_embeddings = custom_text_encoder(tokens.to(device))

tokens = CLIP_tokenizer(
    example_texts, padding="max_length",
    max_length=CLIP_tokenizer.model_max_length, truncation=True,
    return_tensors="pt",
)
text_embeddings = CLIP_text_encoder(tokens.input_ids.to(device))[0]

generated_images = diffusion.reverse_diffusion(
        model=unet_clip, n_images=n_images, n_channels=1, 
        position_encoding_dim=position_encoding_dim,
        position_encoding_function=positional_encoding,
        save_time_steps=save_time_steps, 
        context=text_embeddings, guidance_strength=3,
    )

fig = plt.figure(figsize=(len(save_time_steps), 1.2 * n_images))
for idx in range(n_images):
    image_reverse_diff_traj = generated_images[idx]
    for j in range(len(image_reverse_diff_traj)):
        plt.subplot(n_images, len(image_reverse_diff_traj), 
                    idx * len(image_reverse_diff_traj) + j + 1)
        plt.imshow(image_reverse_diff_traj[j]
                    .permute(1, 2, 0).cpu().numpy(), cmap="gray")
        if j == 5: plt.title(example_texts[idx], fontsize=10)
        plt.axis("off")
plt.savefig(f"fig_10_B2.pdf", bbox_inches="tight")  ### plt.show()